In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath(".."))
from annnet.core.graph import Graph

### Composite indexing

In [4]:

G = Graph()

G.set_vertex_key("name", "sex")

m = G.get_or_create_vertex_by_attrs(name="a", sex="man")
w = G.get_or_create_vertex_by_attrs(name="a", sex="woman")

print(m, w, m == w)   # expect two different ids, False



kv:name='a'|sex='man' kv:name='a'|sex='woman' False


In [6]:
G.set_vertex_key("name")


ValueError: Composite key conflict for ('a',): kv:name='a'|sex='man' vs kv:name='a'|sex='woman'

In [8]:
m = G.get_or_create_vertex_by_attrs(name="a", sex="man")
w = G.get_or_create_vertex_by_attrs(name="a", sex="woman")
print(m, w, m == w)   # same id, True

kv:name='a'|sex='man' kv:name='a'|sex='man' True


### Flexible directionality

In [11]:
G = Graph()
G.set_vertex_key("name","sex")

u = G.get_or_create_vertex_by_attrs(name="a", sex="man")
v = G.get_or_create_vertex_by_attrs(name="a", sex="woman")

# Helper to read signs
def signs(G, eid):
    s,t,_ = G.edge_definitions[eid]
    col = G.edge_to_idx[eid]
    si = G.entity_to_idx[s]; ti = G.entity_to_idx[t]
    M = G._matrix
    return M.get((si,col),0), M.get((ti,col),0)

# Edge-scope policy
e = G.add_edge(u, v,
    flexible={"var":"capacity", "threshold":0.7, "scope":"edge", "above":"s->t", "tie": "undirected"},
    capacity=0.5, weight=1.0)

print("init:", signs(G,e))            # expect (-1, +1)
G.set_edge_attrs(e, capacity=0.9)
print("after:", signs(G,e))           # expect (+1, -1)




init: (-1.0, 1.0)
after: (1.0, -1.0)


In [13]:
# Vertex-scope policy
G2 = Graph()
G2.set_vertex_key("name","sex")
s = G2.get_or_create_vertex_by_attrs(name="a", sex="man")
t = G2.get_or_create_vertex_by_attrs(name="a", sex="woman")
G2.set_vertex_attrs(s, temp=10.0)
G2.set_vertex_attrs(t, temp=20.0)

e2 = G2.add_edge(s, t,
    flexible={"var":"temp","threshold":0.0,"scope":"vertex","above":"s->t","tie":"undirected"},
    weight=1.0)

print("xs<xt:", signs(G2,e2))         # (-1, +1)
G2.set_vertex_attrs(s, temp=25.0)
print("xs>xt:", signs(G2,e2))         # (+1, -1)
G2.set_vertex_attrs(t, temp=25.0)
print("tie  :", signs(G2,e2))         # since "tie":"undirected", it becomes: (+1, +1)

xs<xt: (-1.0, 1.0)
xs>xt: (1.0, -1.0)
tie  : (1.0, 1.0)


### Kivela Multilayers

In [2]:
import sys
import os
sys.path.insert(0, os.path.abspath(".."))
from annnet.core.graph import Graph

In [6]:
G = Graph()
G.set_aspects(["time"], {"time": ["t1","t2"]})
G.add_slice("t1"); G.add_slice("t2")        # populates elem_layers too
G.add_vertex("u", slice="t1")               # adds (u,("t1",)) to VM
G.add_vertex("v", slice="t2")               # adds (v,("t2",))
G.ensure_node_layer_index()
assert G.has_presence("u", ("t1",))
assert not G.has_presence("u", ("t2",))

In [8]:
G.set_aspects(["time","rel"], {"time":["t1","t2"], "rel":["F","A"]})
G.add_vertex("u"); G.add_vertex("v")
G.add_presence("u", ("t1","F")); G.add_presence("v", ("t1","F"))
G.add_presence("u", ("t2","F"))
G.add_intra_edge_nl("u","v", ("t1","F"), weight=2.0)
G.add_coupling_edge_nl("u", ("t1","F"), ("t2","F"), weight=0.5)
A = G.supra_adjacency()  # builds with tupled layers


In [10]:
G = Graph(directed=True)
G.set_aspects(["time","rel"], {"time":["t1","t2"], "rel":["F","A"]})
G.add_vertex("u"); G.add_vertex("v")
G.add_presence("u", ("t1","F")); G.add_presence("v", ("t1","F"))
G.add_presence("u", ("t2","F"))
G.add_intra_edge_nl("u","v", ("t1","F"), weight=2.0)
G.add_coupling_edge_nl("u", ("t1","F"), ("t2","F"), weight=0.5)

A_intra = G.build_intra_block()
A_inter = G.build_inter_block()
A_coup  = G.build_coupling_block()
L_comb  = G.supra_laplacian("comb")
L_norm  = G.supra_laplacian("norm")


In [12]:
# Categorical across "time" for all nodes, per fixed relation:
G.add_categorical_coupling("time", [["t1","t2"]], weight=0.5)

# Filtered diagonal: couple only friendship layers across t1,t2:
G.add_diagonal_coupling_filter({"time":{"t1","t2"}, "rel":{"F"}}, weight=0.5)

# Explicit layer pairs:
G.add_layer_coupling_pairs([ (("t1","F"), ("t2","F")) ], weight=0.5)


1

In [14]:
# Build graph as before...
A = G.supra_adjacency()
tv = G.adjacency_tensor_view()         # 4-index triplets
A2 = G.flatten_to_supra(tv)            # f(tv)
tv2 = G.unflatten_from_supra(A2)       # f^{-1}(A2)
assert A2.shape == A.shape


In [16]:
# RW step
import numpy as np
A = G.supra_adjacency()
P = G.transition_matrix()
p0 = np.ones(A.shape[0]) / A.shape[0]
p1 = G.random_walk_step(p0)

# Diffusion step
x0 = np.random.rand(A.shape[0])
x1 = G.diffusion_step(x0, tau=0.1, kind="comb")

# Spectral probes
lam2, fvec = G.algebraic_connectivity()
vals, vecs = G.k_smallest_laplacian_eigs(3)

# Coupling sweep
omegas = [0.0, 0.1, 1.0, 10.0]
curve = G.sweep_coupling_regime(omegas, metric="algebraic_connectivity")

vecs

array([[ 0.57735027, -0.31970025],
       [ 0.57735027,  0.81049889],
       [ 0.57735027, -0.49079864]])

In [18]:
# Build supra index on desired layers first
G.ensure_node_layer_index()

# Descriptors
P = G.participation_coefficient()   # dict[node] -> [0,1]
V = G.versatility()                 # dict[node] -> normalized versatility

# Multislice modularity scorer (supply a partition over |V_M|)
n = len(G._row_to_nl)
part = list(range(n))               # toy: each node-layer in its own community
Q0 = G.multislice_modularity(part, gamma=1.0, omega=0.5, include_inter=False)